# Figures for Report

In [ ]:
import os
import sys
import glob
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Local modules
import toshiba
import cinterp

In [ ]:
files = glob.glob(os.path.expanduser('~/Downloads/toshiba/*.nc'))
filename = files[0]
range_count = 200

In [ ]:
sweep = toshiba.read(filename, maxgate=range_count)

In [ ]:
# Get the reflectivity and raw range and azimuth angles
z = sweep['z']
r = 1.0e-3 * sweep['rr']
a = sweep['aa']

# Only retain the 360-deg coverage, take out the extra piece
k = next(k for k, ia in enumerate(a[::-1]) if abs(ia > a[0]) < 0.5) 
a = a[:-k] / 180.0 * np.pi
z = sweep['z'][:-k, :]

rr, aa = np.meshgrid(r, a)
xx = rr * np.sin(aa)
yy = rr * np.cos(aa)

In [ ]:
turbines = cinterp.data.toshiba_cells

a_turb = turbines[:, 0] / 180.0 * np.pi
r_turb = turbines[:, 1]
x_turb = r_turb * np.sin(a_turb)
y_turb = r_turb * np.cos(a_turb)

cells = cinterp.pos2cellid(turbines, a / np.pi * 180.0, r)

z_interp = z.copy()

m = np.zeros(z_interp.shape, dtype=bool)
for c in cells:
    m[c[0], c[1]] = True
m1 = cinterp.dilate(m)
m2 = cinterp.dilate(m1)

tags = cinterp.mask2tags(m1)

# Take out the dilated portion
tags[m1 ^ m] = 0

In [ ]:
tmp = cinterp.colormap.zmap()
zmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', tmp[:, :3], N=len(tmp))

tmp = np.array([
    [0.00, 0.75, 1.00],
    [0.00, 0.30, 1.00],
    [0.75, 1.00, 0.00],
    [0.30, 0.80, 0.00],
    [1.00, 0.60, 0.00],
    [1.00, 0.00, 0.00],
    [0.40, 0.15, 1.00],
    [0.70, 0.15, 0.80],
    [0.35, 0.35, 0.35],
    [0.00, 0.75, 1.00],
    [0.00, 1.00, 0.00],
    [0.75, 0.75, 0.75],
    [1.00, 0.60, 0.00],
    [0.40, 0.15, 1.00]
])
imap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', tmp[:, :3], N=len(tmp))

In [ ]:
ntags = np.array(tags, dtype=np.float)
ntags[tags == 0] = np.nan

plt.figure(figsize=(5.25, 4), dpi=200)
plt.pcolormesh(xx, yy, ntags, cmap=imap)
plt.xlim((-1, 6))
plt.ylim((-12.5, -5.5))
plt.clim((0.75, 7.75))
plt.colorbar()
plt.grid()

In [ ]:
z_interp = cinterp.cinterp_tags(z, a, r, tags)

In [ ]:
plt.figure(figsize=(5.25, 4), dpi=200)
plt.pcolormesh(xx, yy, z, cmap=cmap)
plt.clim((-32, 96))
plt.xlim((-1, 6))
plt.ylim((-12.5, -5.5))
plt.colorbar()
plt.grid()

mask = tags == 1
gmask = cinterp.dilate(mask) ^ mask

plt.figure(figsize=(5.25, 4), dpi=200)
plt.pcolormesh(xx, yy, 5 * (mask + 2 * gmask), cmap=cmap)
plt.clim((-32, 96))
plt.xlim((-1, 6))
plt.ylim((-12.5, -5.5))
plt.colorbar()
plt.grid()

plt.figure(figsize=(5.25, 4), dpi=200)
plt.pcolormesh(xx, yy, z_interp, cmap=cmap)
plt.clim((-32, 96))
plt.xlim((-1, 6))
plt.ylim((-12.5, -5.5))
plt.colorbar()
plt.grid()